# A functional burst search and download service

## Setup

In [1]:
import bursts

In [ ]:
safe_url = 'https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_SLC__1SDV_20220716T115019_20220716T115049_044125_054459_71A9.zip'

## Demoing interior objects
These objects will not likely not be accessed by users, but this function demonstrates how the metadata objects are created

In [ ]:
slc = bursts.SLCMetadata(safe_url)

In [ ]:
swath = bursts.SwathMetadata(slc,'vv',swath_index=0)

In [ ]:
burst = bursts.BurstMetadata(swath,burst_index=0)

## Creating burst metadata datasets
Users have the option of creating either a geodatframe, or a STAC catalog. Burst data download is only enabled for the STAC catalog, but the geodataframe is great for in-notebook visualizations. Hopefully, users won't have to create these datasets themselves and will instead use an ASF-provided dataset.

In [ ]:
gdf = bursts.generate_burst_geodataframe([safe_url])

In [ ]:
gdf.explore()

In [ ]:
stac = bursts.generate_burst_stac_catalog([safe_url])
stac.describe()

## Download burst data
Using the STAC catalog, users can select an STAC item, then pass this item to `bursts.edl_download_burst_data`. The data is returned in a 2d numpy array with a complex float data type.

In [ ]:
item = stac.get_item('S1_SLC_20220716T115020_VV_273858_IW1')
item.properties

In [ ]:
item.assets['VV'].href

In [ ]:
burst_data = bursts.edl_download_burst_data(item,'VV')
burst_data.shape